In [1]:
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator

import theano
import theano.tensor as T
import csv

In [2]:
#generate the file we will be using
def generateDataFiles(plot = True, size = 0, w1 = 0, w2 = 0):
    x = T.vector('x')
    w = T.vector('w')

    s = 1 / (1 + T.exp(-T.dot(x,w)))
    logistic = theano.function([x,w], s)
    epsilon = 0.05

    data = open("Data.txt", "w")
    writer = csv.writer(data, delimiter=',')
    
    if size == 0:
        size = 50

    x1 = np.random.uniform(-10,10,size)
    x2 = np.random.uniform(-10,10,size)

    if w1 == 0:
        w1 = np.random.uniform(-2,2,1)[0]
    if w2 == 0:
        w2 = np.random.uniform(-2,2,1)[0]
    print("Actual w1 = %f and w2 = %f\n"%(w1,w2))

    for i in range(0,size):
        val1 = x1[i]
        val2 = x2[i]
        k = logistic([val1,val2],[w1,w2])
        if 1 - k < epsilon:
            writer.writerow([1,val1,val2])
        elif k < epsilon:
            writer.writerow([0,val1,val2])
    data.close()
    if plot:
        plotData(w1,w2)


In [100]:
#calculate Error from training data
def calculateError(w1, w2, display = True):
    x = T.vector('x')
    w = T.vector('w')

    s = 1 / (1 + T.exp(-T.dot(x,w)))
    logistic = theano.function([x,w], s)
    
    if display:
        print("With: w1 = %f and w2 = %f"%(w1,w2))
    

    data = []
    sumError = 0
    sumErrorSquare = 0
    with open('Data.txt') as fp:
        reader = csv.reader(fp, delimiter=',')
        for line in reader:
            data.append([int(line[0]), float(line[1]), float(line[2])])
    if display:
        print('y\t\tf(x)\t\tE\t\tE^2')
    for i in range(0,len(data)):
        x1 = data[i][1]
        x2 = data[i][2]
        f = logistic([x1,x2],[w1,w2])
        e = data[i][0] - f
        e2 = e**2
        sumError += e
        sumErrorSquare += e2
        if display:
            print('%f\t%f\t%f\t%f'%(data[i][0],f,e,e2))
    if display:
        print("\nSum:\t\t\t\t%f\t%f"%(sumError, sumErrorSquare))
    return sumErrorSquare

In [4]:
#plot Data from Data.txt
def plotData(w1,w2):
    data = []
    with open('Data.txt') as fp:
        reader = csv.reader(fp, delimiter=',')
        for line in reader:
            data.append([int(line[0]), float(line[1]), float(line[2])])
    for i in range(0,len(data)):
        if data[i][0] == 1:
            plt.scatter(data[i][1],data[i][2],20, 'r')
        else:
            plt.scatter(data[i][1],data[i][2],20, 'y')
    plt.plot([0,w1],[0,w2])
    plt.show()

In [5]:
#attempt to find my w
def findW(plot = True, showError = True):
    x = T.vector('x')
    w = T.vector('w')

    s = 1 / (1 + T.exp(-T.dot(x,w)))
    logistic = theano.function([x,w], s)

    gs = T.grad(s, w)
    dlogistic = theano.function([x,w], gs)
    

    w1 = np.random.uniform(-2,2,1)[0]
    w2 = np.random.uniform(-2,2,1)[0]
    print("Starting Values: w1 = %f and w2 = %f"%(w1,w2))

    data = []
    with open('Data.txt') as fp:
        reader = csv.reader(fp, delimiter=',')
        for line in reader:
            data.append([int(line[0]), float(line[1]), float(line[2])])
    for kk in range(0,20):
        sumError = 0
        sumErrorSquare = 0
        sumErrorDeriv = 0
        for i in range(0,len(data)):
            x1 = data[i][1]
            x2 = data[i][2]
            f = logistic([x1,x2],[w1,w2])
            e = data[i][0] - f
            e2 = e**2
            sumError += e
            sumErrorSquare = sumErrorSquare + e2
            sumErrorDeriv += 2 * e * dlogistic([x1,x2],[w1,w2])
        w1 = w1 -  0.005 * sumErrorDeriv[0]
        w2 = w2 - 0.005 * sumErrorDeriv[1]
    print("Final Values: w1 = %f and w2 = %f"%(w1,w2))
    if plot:
        plotData(w1,w2)
    if showError:    
        print("\nCalculating Error ... ")
    return calculateError(w1,w2, showError)
    

In [6]:
#testing the the function to find a w
generateDataFiles(False)
for i in range(0,5):
    error = findW(False, False)
    print("The error for these values:",error)
    print("")
    


Actual w1 = 0.737624 and w2 = -0.335892

Starting Values: w1 = -0.856407 and w2 = -1.262587
Final Values: w1 = -1.493738 and w2 = -0.665903
The error for these values: 25.0715000391

Starting Values: w1 = -1.100743 and w2 = 1.967303
Final Values: w1 = -1.615693 and w2 = 1.640753
The error for these values: 27.2117578556

Starting Values: w1 = 1.860659 and w2 = -1.436796
Final Values: w1 = 1.859961 and w2 = -1.437462
The error for these values: 0.000453779820769

Starting Values: w1 = 1.243822 and w2 = 1.541506
Final Values: w1 = 0.596879 and w2 = 1.942579
The error for these values: 14.8670001624

Starting Values: w1 = 1.720538 and w2 = 1.093287
Final Values: w1 = 1.321167 and w2 = 1.579028
The error for these values: 8.64950819074



In [74]:

def generateAndFindWPlotError():
    data = generateGraphData(10)
    f = []
    for i in range(0,len(data[0])):
        #errors.append([w1[i],fixedW2,calculateError(w1[i],fixedW2, False)])
        f.extend([calculateError(data[0][i],data[1][i], False)])
    data.extend([f])
    return data

In [102]:
#testing functions to plot data
data = generateAndFindWPlotError()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(data[0], data[1])
ax.plot_surface(X,Y,data[2])
plt.show()

In [68]:
#used to genereate range of values for w1 and w2
def generateGraphData(numberOfEach):
    x = np.linspace(-3,3,numberOfEach)
    x = np.repeat(x, numberOfEach).tolist()
    y = np.linspace(-3,3,numberOfEach)
    y = np.tile(y,numberOfEach).tolist()
    return [x,y]

In [101]:
#testing validity of the generateAndFindWPlotError function and generateGraphData function

data = generateAndFindWPlotError()
goodData = []
for i in range(0,len(data[0])):
    if data[2][i] < 0.01:
        goodData.extend([[data[0][i], data[1][i]]])
        plotData(data[0][i],data[1][i])
print(goodData)

[[-1.6666666666666667, 3.0], [-1.0, 2.333333333333333], [-1.0, 3.0], [-0.3333333333333335, 1.666666666666666], [-0.3333333333333335, 2.333333333333333], [-0.3333333333333335, 3.0], [0.33333333333333304, 2.333333333333333], [0.33333333333333304, 3.0]]
